<a id="top"></a>
# Tiny YOLO V3 Object Detection Sample

## OpenVINO version check:
You are currently using the latest development version of Intel® Distribution of OpenVINO™ Toolkit. Alternatively, you can open a version of this notebook for the Intel® Distribution of OpenVINO™ Toolkit LTS version by [clicking this link](../../../../openvino-lts/developer-samples/python/tiny-yolo-v3-python/tiny_yolo_v3.ipynb).

## Prerequisites
This sample requires the following:
- All Files are present and in the following directory structure before inference task:
    - **object_detection_demo_yolov3_async.py** - Base Python* code to perform inference
    - **common/monitors.py** - Utility script that handles display functionality
    - **tiny_yolo_v3.ipynb** - Jupyter* Notebook
    - **frozen_tinyyolov3.pb** - Frozen Tensorflow Tiny Yolo V3 model
    - **/data/reference-sample-data/tiny-yolo-v3-python/classroom.mp4** - Test video
    - **coco.names** - Labels file that has all COCO dataset classes the model was trained with

It is recommended that you have already read the following from [Get Started on the Intel® DevCloud for the Edge](https://devcloud.intel.com/edge/home/):
- [Overview of the Intel® DevCloud for the Edge](https://devcloud.intel.com/edge/get_started/devcloud/)
- [Overview of the Intel® Distribution of OpenVINO™ toolkit](https://devcloud.intel.com/edge/get_started/openvino/)

<br><div class=note><i><b>Note: </b>It is assumed that the server this sample is being run on is on the Intel® DevCloud for the Edge which has Jupyter* Notebook customizations and all the required libraries already installed.  If you download or copy to a new server, this sample may not run.</i></div>


## Introduction

This sample application demonstrates how a smart video IoT solution may be created using Intel® hardware and software tools to perform object detection with a pre-trained Tiny YOLO V3 model.  This solution detects any number of objects within a video frame looking specifically for known objects. 

The results for each frame annotate the input video with boxes around detected objects with a label and a probability value.

### Key concepts
This sample application includes an example for the following:
- Application:
  - Video is supported using OpenCV
  - OpenCV is used to draw bounding boxes around detected objects, labels, and other information
  - Visualization of the resulting bounding boxes in the output
- Intel® DevCloud for the Edge:
  - Submitting inference as jobs that are performed on different edge compute nodes (rather than on the development node hosting this Jupyter* notebook)
  - Monitoring job status
  - Viewing results and assessing performance for hardware on different compute nodes
- [Intel® Distribution of OpenVINO™ toolkit](https://software.intel.com/openvino-toolkit):
  - Download DarkNet Tiny YOLO V3 weights and convert them to Tensorflow frozen protobuf format. 
  - Create the necessary Intermediate Representation (IR) files for the inference model using the [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html)
  - Run an inference application on multiple hardware devices using the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html)


## Tiny YOLO V3 Object Detection application

The Tiny YOLO V3 Object Detection application uses the Intel® Distribution of OpenVINO™ toolkit to perform inference on an input video to detect people within each frame.  We will setup, run, and view the results for this application for several different hardware devices (CPU, GPU, etc.) available on the compute nodes within the Intel® DevCloud for the Edge.  To accomplish this, we will be performing the following tasks:

1. Download pre-trained DarkNet Tiny YOLO V3 model weights
2. Convert DarkNet Tiny YOLO V3 model to supported TensorFlow format
3. Convert Tensorflow Tiny YOLO V3 to inference model IR files needed to perform inference
2. Create the job file used to submit running inference on compute nodes
3. Submit jobs for different compute nodes and monitor the job status until complete
4. View results and assess performance 

### How it works
At startup the Tiny YOLO V3 Object Detection application configures itself by parsing the command line arguments.  Once configured, the application loads the specified inference model's IR files into the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html) and runs inference on the specified input image or video to detect people.  When an object is detected, a box is drawn around the person on the output frame.

To run the application on the Intel® DevCloud for the Edge, a job is submitted to an edge compute node with a hardware accelerator such as Intel® HD Graphics GPU and Intel® Movidius™ Neural Compute Stick 2.  After inference on the input is completed, the output is stored in the appropriate `results/<architecture>/` directory.  The results are then viewed within this Jupyter* Notebook using the `videoHTML` video playback utility.

The application and inference code for this sample is already implemented in the Python* file: [`object_detection_demo_yolov3_async.py`](./object_detection_demo_yolov3_async.py) and [`monitors.py`](./common/monitors.py).

The following sections will guide you through configuring and running the application.

### Configuration
The following sections describe all the necessary configuration to run the application.
#### Command line arguments
The application is run from the command line using the following format:
```bash
python3 object_detection_demo_yolov3_async.py <arguments...>
```
The required command line _<arguments...>_ to run the Python* executable [`object_detection_demo_yolov3_async.py`](./people_counter.py) are:
- **-m** - Path to the _.xml_ IR file (created using [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html)) for the inference model
- **-i** - Path to input video file
- **-o** - The path to where the output video file will be stored
- **-d** - Device type to use to run inference (CPU, GPU, MYRIAD)
- **-t** - Probability threshold value for the detection of an object


### Imports
We begin by importing all the Python* modules that will be used within this Jupyter* Notebook to run and display the results of the people counter application on the Intel® DevCloud for the Edge:
- [os](https://docs.python.org/3/library/os.html#module-os) - Operating system specific module (used for file name parsing)
- [time](https://docs.python.org/3/library/time.html#module-time) - Time tracking module (used for measuring execution time)
- [matplotlib.pyplot](https://matplotlib.org/) - pyplot is used for displaying output images
- [sys](https://docs.python.org/3/library/sys.html#module-sys) - System specific parameters and functions
- [qarpo.demoutils](https://github.com/ColfaxResearch/qarpo) - Provides utilities for displaying results and managing jobs from within this Jupyter* Notebook

Run the following cell to import the Python* dependencies needed.

<br><div class=tip><b>Tip: </b>Select a cell and then use **Ctrl+Enter** to run that cell.</div>

In [ ]:
import matplotlib.pyplot as plt
import os
import time
import sys
from qarpo.demoutils import *
from qarpo.model_visualizer_link import *
print('Imported Python modules successfully.')

<br><div class=tip><i><b>Tip: </b>The '!' at the beginning is a special Jupyter* Notebook command that allows you to run shell commands as if you are at a command line. The command below will also work in a terminal (with the '!' removed).</i></div>

In [ ]:
!cp -rf /opt/intel/openvino/deployment_tools/inference_engine/demos/python_demos/common/ .
!cp common/monitors.py .

## Convert Tiny YOLO V3 Darknet Model to TensorFlow Format

The purpose of this section is to convert the [DarkNet Pre-Trained Tiny YOLO V3 model(.weights)](https://pjreddie.com/darknet/yolo/) to frozen TensorFlow format (.pb) which Model Optimizer supports. We will be referencing the [instructions](https://docs.openvinotoolkit.org/latest/openvino_docs_MO_DG_prepare_model_convert_model_tf_specific_Convert_YOLO_From_Tensorflow.html) to convert the Darknet model to TensorFlow format(.pb) in the cells below. 

### Download Tiny YOLO V3 Darknet Model Weights and COCO labels file
<br><div class=note><i><b>Note: </b>This step takes ~1-3 minutes</i></div>

In [ ]:
!curl https://pjreddie.com/media/files/yolov3-tiny.weights > yolov3-tiny.weights

In [ ]:
!curl https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names> coco.names

### Convert weights to frozen Tensorflow Format 

We will clone the tensorflow-yolo-v3 repository to access the convert_weights_pb.py python script that can convert all different types of YOLO and Tiny YOLO models to frozen Tensorflow Protobuf files (.pb)

In [ ]:
!git clone https://github.com/mystic123/tensorflow-yolo-v3.git

In [ ]:
!python3 tensorflow-yolo-v3/convert_weights_pb.py --class_names coco.names --data_format NHWC --weights_file yolov3-tiny.weights --tiny 

To view a graph of the model used in this application, run the cell below then select the link generated:

In [ ]:
showModelVisualizerLink("frozen_darknet_yolov3_model.pb")

### Create the IR files for the inference model

The Intel® Distribution of OpenVINO™ toolkit includes the [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) used to convert and optimize trained models into the Intermediate Representation (IR) model files, and the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html) that uses the IR model files to run inference on hardware devices.  The IR model files can be created from trained models from popular frameworks (e.g. Caffe\*, Tensorflow*, etc.). 



The model must be converted from Tensorflow format to OpenVINO's Intermediate Reperesentation format (pb -> .xml + .bin). Run the following cells to convert`Tiny YOLO V3` model to IR using Model Optimizer. The IR will be stored in the `./models` directory relative to the location of this Jupyter* Notebook. 

Run the following cell to use Model Optimizer to create the `FP16` model IR files:

<br><div class=tip><b>Tip: </b>If you don't specify the --data_type argument by default Model Optimizer will convert the model to **FP32** precision </div>`

In [ ]:
!/opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
--input_model frozen_darknet_yolov3_model.pb \
--transformations_config /opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/yolo_v3_tiny.json \
--data_type FP16 \
--batch 1 \
--output_dir models/tinyyolov3/FP16

Run the following cell to use Model Optimizer to create the `FP32` model IR files:

In [ ]:
!/opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
--input_model frozen_darknet_yolov3_model.pb \
--transformations_config /opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/yolo_v3_tiny.json \
--data_type FP32 \
--batch 1 \
--output_dir models/tinyyolov3/FP32

We will be running the inference model on different hardware devices which have different requirements on the precision of the model (see [Inference Engine Supported Model Formats](https://docs.openvinotoolkit.org/latest/_docs_IE_DG_supported_plugins_Supported_Devices.html#supported_model_formats) for details).  For our purposes, we will focus on the use of the two most common precisions, FP32 and FP16.

### Run inference
The following sections will go through the steps to run our inference application on the Intel® DevCloud for the Edge. 

#### Configure input
For convenience and consistency, in the next cell we set the Python* variable `InputVideo` to the input video file we will be using to run our sample application.

<br><div class=note><i><b>Note: </b>
If you want to use a different input video, change the path in the following cell to the path of the video and run the cell again.
</i></div>

In [ ]:
# Set the path to the input video to use for the rest of this sample
InputVideo = "classroom.mp4"
print(f"Input video file set to:{InputVideo}")

# Set maximum number of inference requests for CPU when using the Async API
NumRequests_CPU = 2
print(f"Number of inference requests for CPU set to:{NumRequests_CPU}")

# Set maximum number of inference requests for CPU when using the Async API
NumRequests_GPU = 4
print(f"Number of inference requests for GPU set to:{NumRequests_GPU}")

# Set maximum number of inference requests for NCS2 when using the Async API
NumRequests_NCS2 = 2
print(f"Number of inference requests for NCS2 set to:{NumRequests_NCS2}")

# Set maximum number of inference requests for HDDL-R when using the Async API
NumRequests_HDDLR = 128
print(f"Number of inference requests for HDDL-R set to:{NumRequests_HDDLR}")

#### Optional exercise: View input without inference

If you are curious to see the input video, run the following cell to view the original video stream used for inference and this sample.

In [ ]:
videoHTML('Test Video', [InputVideo])

#### Create the job file
We will run inference on several different edge compute nodes present in the Intel® DevCloud for the Edge. We will send work to the edge compute nodes by submitting the corresponding non-interactive jobs into a queue. For each job, we will specify the type of the edge compute server that must be allocated for the job.

The job file is a [Bash](https://www.gnu.org/software/bash/) script that serves as a wrapper around the Python* executable of our application that will be executed directly on the edge compute node.  One purpose of the job file is to simplify running an application on different compute nodes by accepting a few arguments and then performing accordingly any necessary steps before and after running the application executable.  

For this sample, the job file we will be using is already written for you and appears in the next cell.  The job file will be submitted as if it were run from the command line using the following format:
```bash
tinyyolov3.sh <output_directory> <device> <fp_precision> <input_file> <threshold>
```
Where the job file input arguments are:
- <*output_directory*> - Output directory to use to store output files
- <*device*> - Hardware device to use (e.g. CPU, GPU, etc.)
- <*fp_precision*> - Which floating point precision inference model to use (FP32 or FP16)
- <*input_file*> - Path to input video file
- <*threshold*> - Probability threshold value for the object detection
- <*numrequest*> - Number of inference request to run in parallel on a stream 
- <*numstreams*> - number of streams to run in parallel for the number of infer requests

Based on the input arguments, the job file will do the following:
- Change to the working directory `PBS_O_WORKDIR` where this Jupyter* Notebook and other files appear on the compute node
- Create the <*output_directory*>
- Choose the appropriate inference model IR file for the specified <*fp_precision*>
- Run the application Python* executable with the appropriate command line arguments

Run the following cell to create the `tinyyolov3.sh` job file.  The [`%%writefile`](https://ipython.readthedocs.io/en/stable/interactive/magics.html#cellmagic-writefile) line at the top will write the cell contents to the specified job file `tinyyolov3.sh`.

In [ ]:
%%writefile tinyyolov3.sh

# Store input arguments: <output_directory> <device> <fp_precision>
#                         <input_file> <threshold> <numrequest> <numstreams> <labels>
OUTPUT_FILE=$1
DEVICE=$2
FP_MODEL=$3
INPUT_FILE=$4
THRESHOLD=$5
NUMREQUEST=$6
NUMSTREAMS=$7
LABELS=coco.names

# The default path for the job is the user's home directory,
#  change directory to where the files are.
cd $PBS_O_WORKDIR

# Make sure that the output directory exists.
mkdir -p $OUTPUT_FILE

# Set inference model IR files using specified precision
MODELPATH=models/tinyyolov3/${FP_MODEL}/frozen_darknet_yolov3_model.xml

# Run the Tiny YOLO V3 object detection code
python3 object_detection_demo_yolov3_async.py -m $MODELPATH \
    -i $INPUT_FILE \
    -o $OUTPUT_FILE \
    -d $DEVICE \
    -t $THRESHOLD \
    -nireq $NUMREQUEST \
    -nstreams $NUMSTREAMS \
    -no_show \
    --labels $LABELS 
    


#### How to submit a job

Now that we have the job script, we can submit jobs to edge compute nodes in the Intel® DevCloud for the Edge.  To submit a job, the `qsub` command is used with the following format:
```bash
qsub <job_file> -N <JobName> -l <nodes> -F "<job_file_arguments>" 
```
We can submit people_counter.sh to several different types of edge compute nodes simultaneously or just one node at a time.

There are three options of `qsub` command that we use for this:
- <*job_file*> - This is the job file we created in the previous step
- `-N` <*JobName*> : Sets name specific to the job so that it is easier to distinguish  between it and other jobs
- `-l` <*nodes*> - Specifies the number and the type of nodes using the format *nodes*=<*node_count*>:<*property*>[:<*property*>...]
- `-F` "<*job_file_arguments*>" - String containing the input arguments described in the previous step to use when running the job file

*(Optional)*: To see the available types of nodes on the Intel® DevCloud for the Edge, run the following cell.

In [ ]:
!pbsnodes | grep compnode | awk '{print $3}' | sort | uniq -c

In the above output from executing the previous cell, the properties describe the node, and the number on the left is the number of available nodes of that architecture.

#### Submit jobs

Each of the cells in the subsections below will submit a job to be run on different edge compute nodes. The output of each cell is the _JobID_ for the submitted job.  The _JobID_ can be used to track the status of the job.  After submission, a job will go into a waiting queue before running once the requested compute nodes become available.

<br><div class=note><i><b>Note: </b>You may submit all jobs at once or one at a time.</i></div> 

<br><div class=tip><b>Tip: </b>**Shift+Enter** will run the cell and automatically move you to the next cell. This allows you to use **Shift+Enter** multiple times to quickly run through multiple cells, including markdown cells.</div>

##### Submit to an edge compute node with Intel® Xeon® Gold 6258R CPU
In the cell below, we submit a job to an edge node with an [Intel® Xeon® Gold 6258R Processor](https://ark.intel.com/content/www/us/en/ark/products/199350/intel-xeon-gold-6258r-processor-38-5m-cache-2-70-ghz.html). The inference workload will run on the CPU.

In [ ]:
#Submit job to the queue
job_id_xeon_cascade_lake = !qsub tinyyolov3.sh -l nodes=1:idc018 -F "results/xeon_cascade_lake/ CPU FP32 {InputVideo} 0.4 {NumRequests_CPU} 1" -N tinyyolov3_cascade
print(job_id_xeon_cascade_lake[0])
id_xeon_cascade_lake = job_id_xeon_cascade_lake[0].split('.')[0]
#Progress indicators
if job_id_xeon_cascade_lake:
    progressIndicator('results/xeon_cascade_lake', 'i_progress_'+id_xeon_cascade_lake+'.txt', "Inference", 0, 101)

##### Submit to an edge compute node with an Intel® CPU
In the cell below, we submit a job to an edge node with an [Intel® Core™ i5-6500TE](https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz.html) processor. The inference workload will run on the CPU.

In [ ]:
#Submit job to the queue
job_id_core = !qsub tinyyolov3.sh -l nodes=1:idc001skl -F "results/core/ CPU FP32 {InputVideo} 0.4 {NumRequests_CPU} 1" -N tinyyolov3_core
print(job_id_core[0])
id_core = job_id_core[0].split('.')[0]
#Progress indicators
if job_id_core:
    progressIndicator('results/core', 'i_progress_'+id_core+'.txt', "Inference", 0, 100)

##### Submit to an edge compute node with Intel® Xeon® CPU
In the cell below, we submit a job to an edge node with an [Intel® Xeon® Processor E3-1268L v5](https://ark.intel.com/products/88178/Intel-Xeon-Processor-E3-1268L-v5-8M-Cache-2-40-GHz.html). The inference workload will run on the CPU.

In [ ]:
#Submit job to the queue
job_id_xeon = !qsub tinyyolov3.sh  -l nodes=1:idc007xv5 -F "results/xeon/ CPU FP32 {InputVideo} 0.4 {NumRequests_CPU} 1" -N tinyyolov3__xeon
print(job_id_xeon[0])
id_xeon = job_id_xeon[0].split('.')[0]
#Progress indicators
if job_id_xeon:
    progressIndicator('results/xeon', 'i_progress_'+id_xeon+'.txt', "Inference", 0, 100)

##### Submit to an edge compute node with Intel® Core CPU and using the integrated Intel® GPU
In the cell below, we submit a job to an edge node with an [Intel® Core i5-6500TE](https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz.html). The inference workload will run on the Intel® HD Graphics 530 GPU integrated with the CPU.

In [ ]:
#Submit job to the queue
job_id_gpu = !qsub tinyyolov3.sh -l nodes=1:idc001skl -F "results/gpu/ GPU FP32 {InputVideo} 0.4 {NumRequests_GPU} 1" -N tinyyolov3_gpu 
print(job_id_gpu[0])
id_gpu = job_id_gpu[0].split('.')[0]
#Progress indicators
if job_id_gpu:
    progressIndicator('results/gpu', 'i_progress_'+id_gpu+'.txt', "Inference", 0, 100)

##### Submit to an edge compute node with Intel® Neural Compute Stick 2
In the cell below, we submit a job to an edge node with an [Intel Core i5-6500te](https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz.html) CPU. The inference workload will run on an [Intel® Neural Compute Stick 2](https://software.intel.com/en-us/neural-compute-stick) installed in this node.

In [ ]:
#Submit job to the queue
job_id_ncs2 = !qsub tinyyolov3.sh -l nodes=1:idc004nc2 -F "results/ncs2/ MYRIAD FP16 {InputVideo} 0.4 {NumRequests_NCS2} 1" -N tinyyolov3_ncs2
print(job_id_ncs2[0])
id_ncs2 = job_id_ncs2[0].split('.')[0]
#Progress indicators
if job_id_ncs2:
    progressIndicator('results/ncs2', 'i_progress_'+id_ncs2+'.txt', "Inference", 0, 100)

##### Submit to an edge compute node with Intel® Atom® and using the integrated Intel® GPU
In the cell below, we submit a job to an edge node with an [Intel® Atom® x7-E3950](https://ark.intel.com/products/96488/Intel-Atom-x7-E3950-Processor-2M-Cache-up-to-2-00-GHz.html) processor. The inference workload will run on the integrated Intel® HD Graphics 505 GPU.

In [ ]:
#Submit job to the queue
job_id_atom = !qsub tinyyolov3.sh -l nodes=1:idc008u2g  -F "results/atom/ GPU FP32 {InputVideo} 0.4 {NumRequests_CPU} 1" -N tinyyolov3_atom
print(job_id_atom[0])
id_atom= job_id_atom[0].split('.')[0]
#Progress indicators
if job_id_atom:
    progressIndicator('results/atom', 'i_progress_'+id_atom+'.txt', "Inference", 0, 100)

##### Submit to an edge compute node with Intel® Vision Accelerator Design with Intel® Movidius™ VPUs
In the cell below, we submit a job to an edge node with an [Intel Core i5-6500te CPU](https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz.html). The inference workload will run on an [Intel® Vision Accelerator Design with Intel® Movidius™ VPUs](https://software.intel.com/openvino-toolkit/hardware#VPU) accelerator installed in this node that has eight VPUs.

In [ ]:
#Submit job to the queue
job_id_hddlr = !qsub tinyyolov3.sh -l nodes=1:idc002mx8 -F "results/hddlr/ HDDL FP16 {InputVideo} 0.4 {NumRequests_HDDLR} 1" -N tinyyolov3_hddlr
print(job_id_hddlr[0])
id_hddlr= job_id_hddlr[0].split('.')[0]
#Progress indicators
if job_id_hddlr:
    progressIndicator('results/hddlr', 'i_progress_'+id_hddlr+'.txt', "Inference", 0, 100)

### Monitor job status

To check the status of the jobs that have been submitted, use the `qstat` command.  The custom Jupyter* Notebook widget `liveQstat()` is provided to display the output of `qstat` with live updates.  

Run the following cell to display the current job status with periodic updates. 

In [ ]:
liveQstat()

You should see the jobs that you have submitted (referenced by the `JobID` that gets displayed right after you submit the jobs in the previous step).
There should also be an extra job in the queue named `jupyterhub-singleuser`: this job is your current Jupyter* Notebook session which is always running.

The `S` column shows the current status of each job: 
- If the status is `Q`, then the job is queued and waiting for available resources
- If ste status is `R`, then the job is running
- If the job is no longer listed, then the job has completed

<br><div class=note><i><b>
Note: The amount of time spent in the queue depends on the number of users accessing the requested compute nodes. Once the jobs for this sample application begin to run, they should take from 1 to 5 minutes each to complete.
</b></i></div>

<br><div class=danger><b>Wait!: </b>Please wait for the inference jobs and video rendering to complete before proceeding to the next step to view results.</div>

### View results

Once the jobs have completed, the queue system outputs the stdout and stderr streams of each job into files with names of the forms <*JobName*>.o<*JobID*> and <*JobName*>.e<*JobID*>, respecitvely.  The *JobName* corresponds to the `-N` option when submitting the job using the `qsub` command.  

The output video file for each job is written to the file `output_{job_id}.mp4` located in the directory `results/<device>` that was specified as the output directory to the job file.  We will now use the `videoHTML()` utility to display the output video files within this Jupyter* notebook.  Calling `videoHTML()` from a Python* cell follows the form:
```python
videoHTML(title, [list_of_video_files], statistics(optional))
```
The parameters are:
- *title* - Title to put at the top of the displayed output
- \[*list_of_video_files*\] - Python* list of video files to display
- *statistics(optional)* - Optional statistics file containing the number of seconds it took to process a number of frames

Run the cells below to display the videos.


##### View results from an Intel® Xeon® Gold 6258R CPU

In [ ]:
videoHTML('Intel® Xeon® Gold 6258R CPU', 
          ['results/xeon_cascade_lake/output_{}.mp4'.format(id_xeon_cascade_lake)],'results/xeon_cascade_lake/stats_{}.txt'.format(id_xeon_cascade_lake))

##### View results from an Intel® CPU

In [ ]:
videoHTML('Intel Core CPU', 
         ['results/core/output_{}.mp4'.format(id_core)],'results/core/stats_{}.txt'.format(id_core))


##### View results from an Intel® Xeon® CPU

In [ ]:
videoHTML('Intel Xeon CPU',
          ['results/xeon/output_{}.mp4'.format(id_xeon)],'results/xeon/stats_{}.txt'.format(id_xeon))

##### View results from an Intel® Core CPU and integrated Intel® GPU

In [ ]:
videoHTML('Intel Core + Intel GPU',
          ['results/gpu/output_{}.mp4'.format(id_gpu)],'results/gpu/stats_{}.txt'.format(id_gpu))

##### View results from an Intel® Neural Compute Stick 2

In [ ]:
videoHTML('Intel CPU + Intel NCS2', 
          ['results/ncs2/output_{}.mp4'.format(id_ncs2)], 'results/ncs2/stats_{}.txt'.format(id_ncs2))

##### View results from an Intel® Atom® and integrated Intel® GPU

In [ ]:
videoHTML('Intel Atom CPU + Intel GPU',
          ['results/atom/output_{}.mp4'.format(id_atom)],'results/atom/stats_{}.txt'.format(id_atom))

##### View results from Intel® Vision Accelerator Design with Intel® Movidius™ VPUs

In [ ]:
videoHTML('Intel VPU HDDL-R',
          ['results/hddlr/output_{}.mp4'.format(id_hddlr)],'results/hddlr/stats_{}.txt'.format(id_hddlr))

### View and assess performance results

The running time of each inference task is recorded in `results/<device>/stats.txt`. Run the cell below to plot the results of all jobs side-by-side. Lower values mean better performance for **Inference Engine Processing Time** and higher values mean better performance for **Inference Engine FPS**. When comparing results, please keep in mind that some architectures are optimized for highest performance, others for low power or other metrics.

In [ ]:
# Create table of <architecture>, <title> for plotting
arch_list = [('xeon_cascade_lake', 'Intel Xeon\nGold\n 6258R\nCPU'),
             ('core', 'Intel Core\ni5-6500TE\nCPU'),
             ('xeon', 'Intel Xeon\nE3-1268L v5\nCPU'),
             ('gpu', ' Intel Core\ni5-6500TE\nHD530/GPU'),
             ('ncs2', 'Intel VPU\ni5-6500TE\nNCS2 VPU'),
             ('atom', 'Intel Atom\nx7-E3950\nHD505/GPU'),
             ('hddlr', 'Intel VPU\n8xVPU\nHDDL-R')]
# For each archtecture in table, create path to stats file or placeholder 
stats_list = []
for arch, a_name in arch_list:
    if 'id_'+arch in vars():
        stats_list.append(('results/'+arch+'/stats_'+vars()['id_'+arch]+'.txt', a_name))
    else:
        stats_list.append(('placeholder'+arch, a_name))
# Plot the execution time from the stats files
summaryPlot(stats_list, 'Architecture', 'Time, seconds', 'Inference Engine Processing Time', 'time')
# Plot the frames per second from the stats files
summaryPlot(stats_list, 'Architecture', 'Frames per second', 'Inference Engine FPS', 'fps')

## Telemetry Dashboard
Once your submitted jobs are completed, run the cells below to view telemetry dashboards containing performance metrics for your model and target architecture.

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view metering dashboard of the last job ran on Intel® Xeon® Gold 6258R CPU</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_xeon_cascade_lake[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view telemetry dashboard of the last job ran on Intel® Core™ i5-6500TE</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_core[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view metering dashboard of the last job ran on Intel® Xeon® CPU</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_xeon[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view metering dashboard of the last job ran on Intel® Core CPU and using the onboard Intel® GPU</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_gpu[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view metering dashboard of the last job ran on IEI Mustang-V100-MX8 ( Intel® Movidius™ Myriad™ X Vision Processing Unit (VPU))</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_hddlr[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view metering dashboard of the last job ran on Intel® NCS 2 (Neural Compute Stick 2)</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_ncs2[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view metering dashboard of the last job ran on UP Squared Grove IoT Development Kit (UP2)</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_atom[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

## Next steps
- [More Jupyter* Notebook Samples](https://devcloud.intel.com/edge/advanced/sample_applications/) - additional sample applications 
- [Jupyter* Notebook Tutorials](https://devcloud.intel.com/edge/get_started/tutorials) - sample application Jupyter* Notebook tutorials
- [Intel® Distribution of OpenVINO™ toolkit Main Page](https://software.intel.com/openvino-toolkit) - learn more about the tools and use of the Intel® Distribution of OpenVINO™ toolkit for implementing inference on the edge


## About this notebook

For technical support, please see the [Intel® DevCloud Forums](https://software.intel.com/en-us/forums/intel-devcloud-for-edge)

<p style=background-color:#0071C5;color:white;padding:0.5em;display:table-cell;width:100pc;vertical-align:middle>
<img style=float:right src="https://devcloud.intel.com/edge/static/images/svg/IDZ_logo.svg" alt="Intel DevCloud logo" width="150px"/>
<a style=color:white>Intel® DevCloud for the Edge</a><br>   
<a style=color:white href="#top">Top of Page</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Intel-DevCloud-for-the-Edge-Usage-Agreement.pdf">Usage Agreement (Intel)</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Colfax_Cloud_Service_Terms_v1.3.pdf">Service Terms (Colfax)</a>
</p>